# Import model

In [1]:
%%capture
%load_ext autoreload
%autoreload 2


import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import holoviews as hv
import csv
import math
import re

from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from IPython.core.display import display

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
minian_path = "."

In [2]:
%%capture
sys.path.insert(0, minian_path)
from minian.utilities import (
    open_minian,
    match_information,
    match_path,
    TaskAnnotation,
)

# Specify data location

In [3]:
# Set up Initial Basic Parameters#
dpath="/N/project/Cortical_Calcium_Image/Miniscope data/06.2022_Second_group/AA017_985237_D7/2022_06_12/16_38_34/Miniscope_2/S4"
mouseID, day, session = match_information(dpath)
mouse_path, video_path = match_path(dpath)
timestamp_path = os.path.join(video_path,"timeStamps.csv")
minian_ds_path = os.path.join(dpath, "minian")
odpath="/N/project/Cortical_Calcium_Image/Miniscope data/data_analysis" #output path
opath = os.path.join(odpath, mouseID, day, session)
n_workers = int(os.getenv("MINIAN_NWORKERS", 4))

# Load Data

In [4]:
cluster = LocalCluster(
    n_workers=n_workers,
    memory_limit="5GB",
    resources={"MEM": 1},
    threads_per_worker=2,
    dashboard_address=":8787",
)
annt_plugin = TaskAnnotation()
cluster.scheduler.add_plugin(annt_plugin)
client = Client(cluster)

/N/u/hafu/Carbonate/.conda/envs/minian/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46286 instead
  warnings.warn(


## Data structure of Minian outputs
Here is the output of Minian. It is an xarray Dataset that contains coordinate labels for different aspects of the data. The way you typically reference the data is the same as you would a Python dictionary. That is, using data['key']. For example, data['A']. See more here: https://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset. 

The most common variables you will be accessing are A, S, and sometimes C. 
data['A'] contains the spatial footprints of the detected neurons. 
data['S'] contains the deconvolved spikes.
data['C'] contains the modeled calcium traces. 

Each of these are arranged such that their first dimension is unit_id. So for example, in order to access the first neuron's spatial footprint you would call data['A'].sel(unit_id=0). To access the second, data['A'].sel(unit_id=1), etc.

In [5]:
# print(minian_ds_path)
data = open_minian(minian_ds_path)
data

<xarray.Dataset>
Dimensions:      (frame: 27000, height: 608, shift_dim: 2, unit_id: 72, width: 608)
Coordinates:
    animal       <U11 'Miniscope_2'
  * height       (height) int64 0 1 2 3 4 5 6 7 ... 601 602 603 604 605 606 607
    session      <U2 'S4'
  * unit_id      (unit_id) int64 0 1 2 3 4 5 6 7 8 ... 64 65 66 67 68 69 70 71
    unit_labels  (unit_id) int64 dask.array<chunksize=(72,), meta=np.ndarray>
  * width        (width) int64 0 1 2 3 4 5 6 7 ... 601 602 603 604 605 606 607
  * frame        (frame) int64 266 267 268 269 270 ... 27262 27263 27264 27265
  * shift_dim    (shift_dim) <U6 'height' 'width'
Data variables:
    A            (unit_id, height, width) float64 dask.array<chunksize=(1, 608, 608), meta=np.ndarray>
    f            (frame) float64 dask.array<chunksize=(50,), meta=np.ndarray>
    max_proj     (height, width) uint8 dask.array<chunksize=(608, 608), meta=np.ndarray>
    b0           (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    c0           (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    S            (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    C            (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    motion       (frame, shift_dim) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    b            (height, width) float64 dask.array<chunksize=(608, 608), meta=np.ndarray>

In [6]:
data.unit_labels.values

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, 22, -1, -1, -1, -1, -1, -1, -1, -1, 31, -1, -1,
       34, -1, -1, 37, 38, -1, -1, -1, -1, -1, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71])

In [7]:
data.unit_id.data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71])

# Fix Coordinates

## Add dimension

In [ ]:
data.expand_dims(.)

## Fix animal and session

In [8]:
data["animal"] = mouseID

In [9]:
data.animal

<xarray.DataArray 'animal' ()>
array('AA017', dtype='<U5')
Coordinates:
    animal   <U5 'AA017'
    session  <U2 'S4'

## Add group and day

In [10]:
data.coords["drug"]="Coke"

In [11]:
data.coords["day"] = day

In [12]:
data

<xarray.Dataset>
Dimensions:      (frame: 27000, height: 608, shift_dim: 2, unit_id: 72, width: 608)
Coordinates:
    animal       <U5 'AA017'
  * height       (height) int64 0 1 2 3 4 5 6 7 ... 601 602 603 604 605 606 607
    session      <U2 'S4'
  * unit_id      (unit_id) int64 0 1 2 3 4 5 6 7 8 ... 64 65 66 67 68 69 70 71
    unit_labels  (unit_id) int64 dask.array<chunksize=(72,), meta=np.ndarray>
  * width        (width) int64 0 1 2 3 4 5 6 7 ... 601 602 603 604 605 606 607
  * frame        (frame) int64 266 267 268 269 270 ... 27262 27263 27264 27265
  * shift_dim    (shift_dim) <U6 'height' 'width'
    drug         <U4 'Coke'
    day          <U2 'D7'
Data variables:
    A            (unit_id, height, width) float64 dask.array<chunksize=(1, 608, 608), meta=np.ndarray>
    f            (frame) float64 dask.array<chunksize=(50,), meta=np.ndarray>
    max_proj     (height, width) uint8 dask.array<chunksize=(608, 608), meta=np.ndarray>
    b0           (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    c0           (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    S            (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    C            (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    motion       (frame, shift_dim) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    b            (height, width) float64 dask.array<chunksize=(608, 608), meta=np.ndarray>

In [13]:
data.frame.data[0]

266

## Merge timestamp

In [14]:
# Could be improved if minian part could be improved
timestamp = pd.read_csv(timestamp_path)
timestamp = timestamp[["Frame Number","Time Stamp (ms)"]]
timestamp = timestamp.rename(columns={"Frame Number":"frame"})
if session == "S1":
    timestamp = timestamp.set_index("frame")
    timestamp = timestamp.iloc[:len(data.frame)]
elif session == "S4":    
    timestamp = timestamp.iloc[-len(data.frame):]
    timestamp["frame"] = np.arange(data.frame.data[0],data.frame.data[-1]+1)
    timestamp = timestamp.set_index("frame")
da_ts = timestamp["Time Stamp (ms)"].to_xarray()
da_ts

<xarray.DataArray 'Time Stamp (ms)' (frame: 27000)>
array([2688182, 2688217, 2688249, ..., 3600042, 3600076, 3600109])
Coordinates:
  * frame    (frame) int64 266 267 268 269 270 ... 27261 27262 27263 27264 27265

In [15]:
data.coords["timeStamp"] = da_ts
data

<xarray.Dataset>
Dimensions:      (frame: 27000, height: 608, shift_dim: 2, unit_id: 72, width: 608)
Coordinates:
    animal       <U5 'AA017'
  * height       (height) int64 0 1 2 3 4 5 6 7 ... 601 602 603 604 605 606 607
    session      <U2 'S4'
  * unit_id      (unit_id) int64 0 1 2 3 4 5 6 7 8 ... 64 65 66 67 68 69 70 71
    unit_labels  (unit_id) int64 dask.array<chunksize=(72,), meta=np.ndarray>
  * width        (width) int64 0 1 2 3 4 5 6 7 ... 601 602 603 604 605 606 607
  * frame        (frame) int64 266 267 268 269 270 ... 27262 27263 27264 27265
  * shift_dim    (shift_dim) <U6 'height' 'width'
    drug         <U4 'Coke'
    day          <U2 'D7'
    timeStamp    (frame) int64 2688182 2688217 2688249 ... 3600076 3600109
Data variables:
    A            (unit_id, height, width) float64 dask.array<chunksize=(1, 608, 608), meta=np.ndarray>
    f            (frame) float64 dask.array<chunksize=(50,), meta=np.ndarray>
    max_proj     (height, width) uint8 dask.array<chunksize=(608, 608), meta=np.ndarray>
    b0           (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    c0           (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    S            (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    C            (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    motion       (frame, shift_dim) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    b            (height, width) float64 dask.array<chunksize=(608, 608), meta=np.ndarray>

## Add minute

In [19]:
data.coords["minute"] = np.floor(data["timeStamp"]/60000)

# Add data variables

## Add behavior data

In [16]:
behavior_data=pd.read_csv(os.path.join(mouse_path, mouseID + "_" + day + "_" + "behavior_ms.csv"))
behavior_data=behavior_data.rename(columns={"Frame Number":"frame"})
behavior_data=behavior_data.set_index("frame")
behavior=behavior_data[["reinforcement", "ALP", "IALP"]]
da_reinforcement=behavior["reinforcement"].to_xarray()
da_ALP=behavior["ALP"].to_xarray()
da_IALP=behavior["IALP"].to_xarray()
data["reinforcement"]=da_reinforcement
data["ALP"] = da_ALP
data["IALP"] = da_IALP
data

<xarray.Dataset>
Dimensions:        (frame: 27000, height: 608, shift_dim: 2, unit_id: 72, width: 608)
Coordinates:
  * frame          (frame) int64 266 267 268 269 270 ... 27262 27263 27264 27265
    animal         <U5 'AA017'
  * height         (height) int64 0 1 2 3 4 5 6 ... 601 602 603 604 605 606 607
    session        <U2 'S4'
  * unit_id        (unit_id) int64 0 1 2 3 4 5 6 7 8 ... 64 65 66 67 68 69 70 71
    unit_labels    (unit_id) int64 dask.array<chunksize=(72,), meta=np.ndarray>
  * width          (width) int64 0 1 2 3 4 5 6 7 ... 601 602 603 604 605 606 607
  * shift_dim      (shift_dim) <U6 'height' 'width'
    drug           <U4 'Coke'
    day            <U2 'D7'
    timeStamp      (frame) int64 2688182 2688217 2688249 ... 3600076 3600109
Data variables:
    A              (unit_id, height, width) float64 dask.array<chunksize=(1, 608, 608), meta=np.ndarray>
    f              (frame) float64 dask.array<chunksize=(50,), meta=np.ndarray>
    max_proj       (height, width) uint8 dask.array<chunksize=(608, 608), meta=np.ndarray>
    b0             (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    c0             (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    S              (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    C              (unit_id, frame) float64 dask.array<chunksize=(1, 27000), meta=np.ndarray>
    motion         (frame, shift_dim) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    b              (height, width) float64 dask.array<chunksize=(608, 608), meta=np.ndarray>
    reinforcement  (frame) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    ALP            (frame) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    IALP           (frame) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0

# Save Data

In [24]:
data.minute.data

array([44., 44., 44., ..., 60., 60., 60.])

In [25]:
type(data.frame)

xarray.core.dataarray.DataArray

In [26]:
data.to_zarr(store = opath, mode = 'w')

# Close cluster

In [27]:
client.close()
cluster.close()